# BDF Introduction

The Jupyter notebook for this demo can be found in:
   - docs/quick_start/demo/bdf_demo.ipynb
   - https://github.com/SteveDoyle2/pyNastran/tree/master/docs/quick_start/demo/bdf_demo.ipynb

Import pyNastran

In [52]:
import os
import pyNastran
print (pyNastran.__file__)
print (pyNastran.__version__)
pkg_path = pyNastran.__path__[0]

from pyNastran.bdf.bdf import BDF, read_bdf
from pyNastran.utils import object_attributes, object_methods

print("pkg_path = %s" % pkg_path)

c:\nasa\m4\formats\git\pynastran_1.4\pyNastran\__init__.py
1.4.0
pkg_path = c:\nasa\m4\formats\git\pynastran_1.4\pyNastran


## Loading a BDF
There are two ways to load a BDF; the long way or the short way.

The short way instantiates the **```BDF```** class and the short way uses the **```read_bdf```** function.
As this demo was written for the Jupyter Notebook, we'll use **``read_bdf``** and then mention the other method.  The class-based method allows finer control over things like:
 - what cards should be loaded
 - OpenMDAO dynamic syntax support

### The class-based method

In [53]:
bdf_filename = os.path.abspath(os.path.join(pkg_path, '..', 'models', 'iSat', 'ISat_Launch_Sm_Rgd.dat'))
print(bdf_filename)

# create the BDF object
bdf = BDF()

# read the file from the GUI
# don't cross-reference
bdf.read_bdf(bdf_filename, xref=False)

c:\nasa\m4\formats\git\pynastran_1.4\models\iSat\ISat_Launch_Sm_Rgd.dat


### The function-based method

In [54]:
bdf = read_bdf(bdf_filename, xref=False)

For simplicity of using the demo, we'll again use the ```read_bdf``` method

In [55]:
#bdf_filename = r'D:\work\pynastran_0.8.0_py27\models\iSat\ISat_Launch_Sm_Rgd.dat'
bdf_filename = os.path.abspath(os.path.join(pkg_path, '..', 'models', 'iSat', 'ISat_Launch_Sm_Rgd.dat'))

# read the file as a path
bdf_xref = read_bdf(bdf_filename, xref=True)

We can use the generic object attributes/methods functions

#### Some other very handy methods that will be used later by ```test_bdf```

In [56]:
print(bdf.get_bdf_stats())
print("card_count = %s\n" % bdf.card_count)
print("reject_count = %s" % bdf.reject_count)

---BDF Statistics---
SOL 103

bdf.spcs[1]
  SPC:     1

bdf.params
  PARAM    : 8

bdf.nodes
  GRID     : 5380

bdf.elements
  CBAR     : 827
  CBUSH    : 104
  CHEXA    : 25
  CQUAD4   : 4580
  CTRIA3   : 32

bdf.rigid_elements
  RBE2     : 44

bdf.properties
  PBAR     : 1
  PBARL    : 18
  PBUSH    : 2
  PSHELL   : 8
  PSOLID   : 4

bdf.masses
  CONM2    : 15

bdf.materials
  MAT1     : 14
  MAT8     : 8

bdf.coords
  CORD2R   : 75

bdf.methods
  EIGRL    : 1

bdf.usets
  USET     : 1


card_count = {'ENDDATA': 1, 'PARAM': 8, 'SPC': 1, 'USET': 1, 'EIGRL': 1, 'CORD2R': 75, 'GRID': 5380, 'CQUAD4': 4580, 'CBAR': 827, 'CHEXA': 25, 'RBE2': 44, 'CTRIA3': 32, 'CBUSH': 104, 'CONM2': 15, 'MAT1': 14, 'MAT8': 8, 'PSHELL': 8, 'PBARL': 18, 'PSOLID': 4, 'PBAR': 1, 'PBUSH': 2}

reject_count = {}


## Cross-referencing

Cross-referencing a BDF allows improved usability of the **``BDF``** class.  It comes with some negative side effects, but in general is a very useful thing.  It dramatically minimizes the amount of code you need to write, greatly simplifies future operations, and is highly recommended.

The major downside is it slows down the code.

### Without Cross-Referencing (xref=False)
Here the raw values of the the data objects are returned to us

In [57]:
cquad = bdf.elements[1]
print(cquad)
nid1 = cquad.nodes[0]
print("nid1 = %s" % nid1)
n1 = bdf.nodes[nid1]
cd4 = n1.cd
c4 = bdf.coords[cd4]
print("i (xref=False) = %s" % str(c4.i))
#print object_attributes(c4)

$*
$*  ELEMENT CARDS
$*
CQUAD4         1       1       1       2       4       3

nid1 = 1
i (xref=False) = [1. 0. 0.]


### Cross-Referenced (xref=True)
Here we can trace the referenced objects very easily.

A cross-referenced attribute is indicated with the **``*_ref``** suffix:
  * ``cquad4_element.nodes`` : not cross referenced
  * ``cquad4_element.nodes_ref`` : cross referenced

In [58]:
print("i (xref=True) = %s" % bdf_xref.elements[1].nodes_ref[0].cd_ref.i)

i (xref=True) = [1. 0. 0.]


So how is this done?

In [59]:
cquad.nodes_ref = []
cquad.nodes_ref.append(n1)
print(cquad.nodes_ref[0])

$*
$*  GRID CARDS
$*
GRID           1       4    -4.5    -7.5    -14.       4



#### Let's show off the GRID card

In [60]:
# some Grid methods
n1 = bdf_xref.nodes[1]
print(n1)

# the comment
c1 = bdf_xref.nodes[1].comment
c2 = bdf_xref.nodes[2].comment
print("c1=%r" % c1)
print("c2=%r" % c2)


# get the position of a node
# in the local cooordinate system
print("xyz = %s" % n1.xyz)

# in the global frame
print("position = %s" % n1.get_position())

# in an arbitrary frame
print("wrt5 = %s" % n1.get_position_wrt(bdf, 5))
print("wrt4 = %s" % n1.get_position_wrt(bdf, 4))


$*
$*  GRID CARDS
$*
GRID           1       4    -4.5    -7.5    -14.       4

c1='$*\n$*  GRID CARDS\n$*\n'
c2=''
xyz = [ -4.5  -7.5 -14. ]
position = [ -4.5  -7.5 -14. ]
wrt5 = [  2.12132034  14.         -26.59188309]
wrt4 = [ -4.5  -7.5 -14. ]


Now let's modify the **``GRID``** card and write it out

In [61]:
n1 = bdf_xref.nodes[1]
n1.xyz[1] = -7.5
print("repr  = %s" % n1.repr_fields())
print("raw   = %s" % n1.raw_fields())

#n1.xyz[1] = 100000000000.
print("repr2 = %s" % n1.repr_fields())
print(n1)
print(n1.write_card(size=8))
print(n1.write_card(size=16, is_double=False))
print(n1.write_card(size=16, is_double=True))

repr  = ['GRID', 1, 4, -4.5, -7.5, -14.0, 4, '', None]
raw   = ['GRID', 1, 4, -4.5, -7.5, -14.0, 4, '', 0]
repr2 = ['GRID', 1, 4, -4.5, -7.5, -14.0, 4, '', None]
$*
$*  GRID CARDS
$*
GRID           1       4    -4.5    -7.5    -14.       4

$*
$*  GRID CARDS
$*
GRID           1       4    -4.5    -7.5    -14.       4                

$*
$*  GRID CARDS
$*
GRID*                  1               4            -4.5            -7.5
*                   -14.               4                                

$*
$*  GRID CARDS
$*
GRID*                  1               4-4.500000000D+00-7.500000000D+00
*       -1.400000000D+01               4                                



### Examples of xref on elements

In [62]:
eid100 = bdf_xref.elements[100]
print(eid100)
print("nodes = %s" % eid100.nodes)
print("--node0--\n%s" % eid100.nodes_ref[0])
print("--cd--\n%s" % eid100.nodes_ref[0].cd)
print("cd.cid = %s" % eid100.nodes_ref[0].cd_ref.cid)

print("area = %s" % eid100.Area())
print("mass = %s" % eid100.Mass())
print("--pid--\n%s" % eid100.pid)
print("pid.pid = %s" % eid100.pid_ref.pid)
print("pid.Pid() = %s" % eid100.Pid())

print(eid100.pid_ref.mid1_ref)
print("type = %s" % eid100.pid_ref.mid1_ref.type)
print("nu12 = %s" % eid100.pid_ref.mid1_ref.nu12)
print("mass = %s" % eid100.Mass())

CQUAD4       100       1     149     152     161     160

nodes = [149, 152, 161, 160]
--node0--
GRID         149       4      3.     7.5   -16.5       4

--cd--
4
cd.cid = 4
area = 3.75
mass = 3.642880307396999e-05
--pid--
1
pid.pid = 1
pid.Pid() = 1
$*
$*  I-DEAS Material: 6  name: BUS_CFRP_PW_ORTHO
$* M46J PW ETW
MAT8           6   1.7+7   1.7+7     .98 340000. 180000. 180000..0001712
                           71.33

type = MAT8
nu12 = 0.98
mass = 3.642880307396999e-05


## Write the modified deck
Let's first switch to the desktop to make the file easy to find

In [63]:
import getpass
name = getpass.getuser()
os.chdir(os.path.join(r'C:\Users', name, 'Desktop'))

In [64]:
pwd

'C:\\Users\\sdoyle\\Desktop'

There are two ways to write a deck
 - **``interspersed``** : alternate properties and elements (similar to how Patran writes decks)
 - **``not-interspersed (default)``** : much faster
 
We can also use 8 or 16 character field width as well as double precision.

Note that double precision only works for certain cards (e.g. ``GRID``, ``COORD``, ``DMIG``) and not much else.

In [65]:
bdf_xref.write_bdf('fem.bdf', interspersed=False, size=8, is_double=False)
!tail -n 5 "fem.bdf"

bdf_xref.write_bdf('fem.bdf', interspersed=True, size=16, is_double=False)
!tail "fem.bdf"

bdf_xref.write_bdf('fem.bdf', interspersed=True, size=16, is_double=True)
!tail "fem.bdf"

CORD2R        75        1.355-13-2.19-15    -40.1.355-13-2.19-15      0.
             40.-2.19-15    -40.
CORD2R        76        1.355-13-2.19-15    -40.1.355-13-2.19-15      0.
             40.-2.19-15    -40.


ENDDATA
*
CORD2R*               75                 1.3549966049-13-2.1854783949-15
*                   -40. 1.3549966049-13-2.1854783949-15              0.
*                    40.-2.1854783949-15            -40.
*
CORD2R*               76                 1.3549966049-13-2.1854783949-15
*                   -40. 1.3549966049-13-2.1854783949-15              0.
*                    40.-2.1854783949-15            -40.
*
ENDDATA


*
CORD2R*               75                1.3549966049D-13-2.185478395D-15
*       -4.000000000D+011.3549966049D-13-2.185478395D-150.0000000000D+00
*       4.0000000000D+01-2.185478395D-15-4.000000000D+01
*
CORD2R*               76                1.3549966049D-13-2.185478395D-15
*       -4.000000000D+011.3549966049D-13-2.185478395D-150.0000000000D+00
*       4.0000000000D+01-2.185478395D-15-4.000000000D+01
*
ENDDATA


In [66]:
bdf_filename

'c:\\nasa\\m4\\formats\\git\\pynastran_1.4\\models\\iSat\\ISat_Launch_Sm_Rgd.dat'